# What is decorator?

## Decorator is a single argument callable object

```python
def decorator(func):
    # do something
    return ...

@decorator
def a_func(*args, **kwargs):
    return ..

result = a_func(*args, **kwargs)
```

is actuall doing:

```python
def decorator(func):
    # do something
    return ...

def a_func(*args, **kwargs):
    return ..

result = decorator(a_func)(*args, **kwargs)
```

**Any object (usually be a function) taking a function object as the only argument, and returns another function** can be ``Decorator``.

## Decorator function been executed when it is been used

The following two syntax are the **same**.

In [1]:
from __future__ import print_function


def print_result(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print("result is {}".format(result))
        return result
    return wrapper

In [14]:
# syntax 1

@print_result
def add_two(a, b):
    return a + b

add_two(1, 2)

result is 3


3

In [15]:
# syntax 2

def add_two(a, b):
    return a + b

add_two = print_result(add_two)

add_two(1, 2)

result is 3


3

In the second case, function ``add_two`` are changed before it is been called. In other word, in the first case, function ``add_two`` are also changed at the moment when it is been defined. The ``add_two = print_result(add_two)`` **has been executed once before it is been actually used**.

## A class can be a decorator too

Since any callable object can be a decorator, a class can be decorator too.

In [26]:
from __future__ import print_function

class PrintResult(object):
    def __init__(self, func):
        self._func = func
    
    def __call__(self, *args, **kwargs):
        result = self._func(*args, **kwargs)
        print("result is {}".format(result))
        return result
    
@PrintResult
def add_two(a, b):
    return a + b

In [27]:
add_two(1, 2)

result is 3


3

What happened under the hood is:

```python
def add_two(a, b):
    return a + b

instance_print_result = PrintResult(add_two) # now add_two is not a function anymore, it is an instance of ``PrintResult``.
add_two = instance_print_result

add_two(1, 2) # instance_print_result.__call__(1, 2)
```

It is little bit harder to apply decorator to both simple function and **a method**.

In [1]:
from __future__ import print_function
import types


class PrintResult(object):
    def __init__(self, func):
        self._func = func

    def __call__(self, *args, **kwargs):
        result = self._func(*args, **kwargs)
        print("result is {}".format(result))
        return result

    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            return types.MethodType(self, instance)


class Calculator(object):
    @PrintResult
    def add_two(self, a, b):
        return a + b


cal = Calculator()
cal.add_two(1, 2)

result is 3


3

If we don't define ``PrintResult.__get__``, let's see what gonna happen:

```python
class Calculator(object):
    def add_two(self, a, b):
        return a + b

Calculator.add_two = PrintResult(Calculator.add_two)

cal = Calculator()
cal.add_two(1, 2)
```

``cal.add_two`` now is an instance of ``PrintResult``. ``cal.add_two._func`` is actually ``Calculator.add_two``, which has three arguments, and the first one is the ``self`` to represent ``Calculator`` instance. So the ``cal.add_two(1, 2)`` is actually doing ``cal.add_two._func``, which is ``Calculator.add_two(1, 2)`` **watch close, it is not** ``Calculator().add_two(1, 2)``. **It requires 3 argument but it only get two**!

```python
class PrintResult(object):
    ...
    
    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            return types.MethodType(self, instance)
```

By adding a ``PrintResult.__get__(self, instance, cls)`` method, when you do ``cal.add_two``, it first invokes the descriptor protocal, and returns ``PrintResult(Calculator.add_two).__get__(self, cal, Calculator)`` and 


In [11]:
from __future__ import print_function
import types


class PrintResult(object):
    def __init__(self, func):
        self._func = func

    def __call__(self, *args, **kwargs):
        result = self._func(*args, **kwargs)
        print("result is {}".format(result))
        return result

    def __get__(self, instance, cls):
        print(self, instance, cls)
        if instance is None:
            return self
        else:
            return types.MethodType(self, instance)

# Everything above is exactly the same as the previous one

class Calculator(object):
    def add_two(self, a, b):
        return a + b

Calculator.add_two = PrintResult(Calculator.add_two)

cal = Calculator()
cal.add_two(1, 2)

<__main__.PrintResult object at 0x109548ed0> <__main__.Calculator object at 0x109b27510> <class '__main__.Calculator'>
result is 3


3